Tương tự như Averaging, blending là một quá trình lấy trung bình các trọng số, trong đó các trọng số được sử dụng để kết hợp các predictions được ước tính trên holdout set và meta-model được training trên đó. Một meta-model đơn giản là một thuật toán ML có thể học từ output của một model khác. Thông thường, meta-model là một linear model.

Quy trình của Blending bao gồm:
1. Trước khi bắt đầu xây dựng model, chúng ta sẽ trích ngẫu nhiên một
holdout set từ training data. Thông thường, holdout set là khoảng 10%
training set, tuy nhiên, tùy thuộc vào hoàn cảnh chúng ta hoàn toàn có thể biến tấu holdout set này, nó có thể ít hơn cũng như nhiều hơn. Trong quá trình lấy mẫu (sampling), chúng ta có thể thực hiện lấy mẫu theo từng tầng dữ liệ (có thể hiểu là k-fold lấy từng fold) để đảm bảo tính đại diện của mẫu.
2. Train tất cả model trên training set
3. Thực hiện quá trình predict trên holdout set và test set.
4. Sử dụng holdout predictions làm training data để train meta-model và sử dụng meta-model này để thực hiện quá trình predict trên test set. Ngoài ra, chúng ta có thể sử dụng meta-model và các model trước đó để áp dụng các cơ chế ensemble đã được học.

In [ ]:
!git clone https://github.com/anminhhung/small_dog_cat_dataset

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [14]:
import glob 
import os 
import cv2 
import numpy as np 

from skimage.feature import hog

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

time: 1.13 ms (started: 2023-06-08 04:13:55 +00:00)


In [2]:
def read_file(path, target_size=(64, 64)):
    datas = []
    label = []

    for category in os.listdir(path):
     category_dir = os.path.join(path, category)
     for image_name in os.listdir(category_dir):
        image_path = os.path.join(category_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, target_size)
        datas.append(image)
        label.append(category)

    return np.array(datas), np.array(label)

In [7]:
train_dir = 'small_dog_cat_dataset/train/'
test_dir = 'small_dog_cat_dataset/test/'
target_size = (64,64)

train_data, train_label = read_file(train_dir, target_size)
test_data, test_label = read_file(test_dir, target_size)

time: 8.45 s (started: 2023-06-08 04:10:29 +00:00)


In [8]:
len(train_data), len(train_label), len(test_data), len(test_label)

(2000, 2000, 600, 600)

time: 4.32 ms (started: 2023-06-08 04:10:38 +00:00)


In [9]:
# feature selection 
def hog_feature(data):
   hog_gray_features = []

   for image in data:
      gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
      hog_features, _ = hog(gray_image, visualize=True,
                               block_norm='L2-Hys',
                               pixels_per_cell=(16, 16),
                                cells_per_block=(2, 2))
      hog_gray_features.append(hog_features)

   return np.array(hog_gray_features)

time: 12.9 ms (started: 2023-06-08 04:11:53 +00:00)


In [10]:
train_data = hog_feature(train_data)
test_data = hog_feature(test_data)

time: 12.9 s (started: 2023-06-08 04:11:58 +00:00)


In [11]:
# K-fold validation 
kf = KFold(n_splits=5, shuffle=True, random_state=0)

time: 581 µs (started: 2023-06-08 04:12:11 +00:00)


In [12]:
model_1 = SVC(probability=True, random_state=0)
model_2 = RandomForestClassifier(random_state=0)
model_3 = KNeighborsClassifier()

time: 774 µs (started: 2023-06-08 04:12:11 +00:00)


In [16]:
X_blend, X_holdout, y_blend, y_holdout = train_test_split(train_data, train_label, test_size=0.3, random_state=42)
model_1.fit(X_blend, y_blend)
model_2.fit(X_blend, y_blend)
model_3.fit(X_blend, y_blend)

proba = np.stack(
    [model_1.predict_proba(X_holdout)[:, 1],
    model_2.predict_proba(X_holdout)[:, 1],
    model_3.predict_proba(X_holdout)[:, 1]]
).T

scaler = StandardScaler()
proba = scaler.fit_transform(proba)

time: 6.31 s (started: 2023-06-08 04:14:42 +00:00)


In [17]:
# train meta-model 
from sklearn.linear_model import LogisticRegression
blender = LogisticRegression(solver='liblinear')
blender.fit(proba, y_holdout)
print(blender.coef_) # kiểm tra hệ số 

[[0.76139929 0.59833626 0.13685654]]
time: 15.8 ms (started: 2023-06-08 04:15:13 +00:00)


In [19]:
# predict 
test_proba = np.stack(
    [model_1.predict_proba(test_data)[:, 1],
    model_2.predict_proba(test_data)[:, 1],
    model_3.predict_proba(test_data)[:, 1]]
).T

blending = blender.predict_proba(test_proba)[:, 1]
ras = roc_auc_score(y_true=test_label, y_score=blending)
print(f"ROC-AUC for linear blending {blender} is: {ras:0.5f}")

ROC-AUC for linear blending LogisticRegression(solver='liblinear') is: 0.80643
time: 377 ms (started: 2023-06-08 04:23:45 +00:00)
